In [2]:
import os

In [3]:
import json
import pandas as pd
import traceback

In [4]:
from langchain.chat_models import ChatOpenAI

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
KEY = os.getenv("OPENAI_API_KEY")

In [7]:
llm = ChatOpenAI(
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=KEY,
    model_name="mistralai/mistral-7b-instruct",
    temperature=0
)


C:\Users\Ranjeet Kumar\AppData\Local\Temp\ipykernel_1452\2679247772.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(


In [8]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000186FC8772E0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000186FC88F160>, model_name='mistralai/mistral-7b-instruct', temperature=0.0, openai_api_key='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', openai_api_base='https://openrouter.ai/api/v1', openai_proxy='')

In [9]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [10]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },  
}

In [11]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [12]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
)

In [13]:
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = "quiz", verbose = True)

C:\Users\Ranjeet Kumar\AppData\Local\Temp\ipykernel_1452\3462200221.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = "quiz", verbose = True)


In [14]:
TEMPLATE2 = """
You are an expert english grammerian and writer. Given a Multiple choice Quiz for {subject} students.\
You need to evaluate the complexity of question and give a complete analysis of quiz. Only use at max 50 words for complexity.qqq
If the Quiz is not at per with the congnitive and analytical abalities of the students, \
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abalities.
Quiz_MCQs:
{quiz}

Check from an Expert English Writer of the above quiz:
"""

In [15]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables = ["subject", "quiz"],
    template = TEMPLATE
)

In [16]:
review_chain = LLMChain(llm = llm, prompt = quiz_evaluation_prompt, output_key = "review", verbose = True)

In [17]:
generate_evaluate_chain = SequentialChain(chains = [quiz_chain, review_chain], input_variables = ["text", "number", "subject", "tone", "response_json"],
                                          output_variables = ["quiz", "review"], verbose = True)

In [18]:
file_path = r"C:\Users\Ranjeet Kumar\Desktop\mcqgen\data.txt"

In [19]:
file_path

'C:\\Users\\Ranjeet Kumar\\Desktop\\mcqgen\\data.txt'

In [20]:
with open(file_path, "r") as file:
    TEXT = file.read()

In [21]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contributed 

In [22]:
# Serialize the python dict into a JSON format string

json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [23]:
NUMBER = 5
SUBJECT = "machine learning"
TONE = "simple"

In [24]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

C:\Users\Ranjeet Kumar\AppData\Local\Temp\ipykernel_1452\861068774.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = generate_evaluate_chain(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who

In [25]:
print(f"Total token : {cb.total_tokens}")
print(f"Prompt token : {cb.prompt_tokens}")
print(f"Completion token : {cb.completion_tokens}")
print(f"Total cost : {cb.total_cost}")

Total token : 3048
Prompt token : 1882
Completion token : 1166
Total cost : 0.0


In [26]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems

In [30]:
quiz = response.get("quiz")

In [33]:
json.loads(quiz)

{'1': {'mcq': "Who coined the term 'machine learning' in 1959?",
  'options': {'a': 'Alan Turing',
   'b': 'Arthur Samuel',
   'c': 'Donald Hebb',
   'd': 'Warren McCulloch'},
  'correct': 'b'},
 '2': {'mcq': 'Which of the following books was published in 1949 and introduced a theoretical neural structure formed by certain interactions among nerve cells?',
  'options': {'a': 'The Organization of Behavior',
   'b': 'Learning Machines',
   'c': 'Computing Machinery and Intelligence',
   'd': 'Pattern Recognition'},
  'correct': 'a'},
 '3': {'mcq': "Which of the following was developed by Raytheon Company in the early 1960s and was an experimental 'learning machine' with punched tape memory?",
  'options': {'a': 'Cybertron',
   'b': 'Cybernetics',
   'c': 'Cognition',
   'd': 'Computation'},
  'correct': 'a'},
 '4': {'mcq': 'What is the definition of machine learning given by Tom M. Mitchell?',
  'options': {'a': 'A computer program is said to learn from experience E with respect to some 

In [34]:
quiz = json.loads(quiz)

In [36]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "choice": options, "correct": correct})

In [37]:
quiz_table_data

[{'MCQ': "Who coined the term 'machine learning' in 1959?",
  'choice': 'a: Alan Turing | b: Arthur Samuel | c: Donald Hebb | d: Warren McCulloch',
  'correct': 'b'},
 {'MCQ': 'Which of the following books was published in 1949 and introduced a theoretical neural structure formed by certain interactions among nerve cells?',
  'choice': 'a: The Organization of Behavior | b: Learning Machines | c: Computing Machinery and Intelligence | d: Pattern Recognition',
  'correct': 'a'},
 {'MCQ': "Which of the following was developed by Raytheon Company in the early 1960s and was an experimental 'learning machine' with punched tape memory?",
  'choice': 'a: Cybertron | b: Cybernetics | c: Cognition | d: Computation',
  'correct': 'a'},
 {'MCQ': 'What is the definition of machine learning given by Tom M. Mitchell?',
  'choice': 'a: A computer program is said to learn from experience E with respect to some class of tasks T and performance measure P if its performance at tasks in T, as measured by P

In [39]:
quiz = pd.DataFrame(quiz_table_data)

In [40]:
quiz.to_csv("machine_learning_quiz.csv", index = False)